In [1]:
%pip install langgraph-checkpoint-sqlite

  Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached ormsgpack-1.12.2-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl (46 kB)
   ---------------------------------------- 0.0/281.5 kB ? eta -:--:--
   ---------------- ----------------------- 112.6/281.5 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 281.5/281.5 kB 4.3 MB/s eta 0:00:00
Using cached ormsgpack-1.12.2-cp311-cp311-win_amd64.whl (117 kB)
  Attempting uninstall: ormsgpack
    Found existing installation: ormsgpack 1.10.0
    Uninstalling ormsgpack-1.10.0:
      Successfully uninstalled ormsgpack-1.10.0
  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 2.1.1
    Uninstalling langgraph-checkpoint-2.1.1:
      Successfully uninstalled langgraph-checkpoint-2.1.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.6.8 requires langgraph-checkpoint<3.0.0,>=2.1.0, but you have langgraph-checkpoint 4.0.0 which is incompatible.
langgraph-prebuilt 0.6.4 requires langgraph-checkpoint<3.0.0,>=2.1.0, but you have langgraph-checkpoint 4.0.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\nages\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
import sqlite3

from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI()

In [2]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
def chat_node(state: ChatState):
    response = llm.invoke(state['messages'])
    return {'messages': [response]}

In [4]:
#checkpointer
conn = sqlite3.connect(database='chatbot.db', check_same_thread=False)
checkpointer = SqliteSaver(conn = conn)

#graph
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

In [5]:
chatbot = graph.compile(checkpointer=checkpointer)

In [7]:
while True:
    user_input = input("You: ")
    print("user:", user_input)
    if user_input.lower() == "exit":
        break
    response = chatbot.invoke(
        {'messages': [HumanMessage(content=user_input)]},
        config = {"configurable": {"thread_id": "thread-4"}}
    )
    print("Chatbot:", response['messages'][0].content)

user: hi
Chatbot: hi
user: my name is nagesh mashette.
Chatbot: hi
user: hi
Chatbot: hi
user: my name is nagesh mashette.
Chatbot: hi
user: hello
Chatbot: hi
user: exit
